In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml

In [2]:
import seaborn as sns
sns.set_style("whitegrid")

In [3]:
import os

In [4]:
# DIR = "../multirun/2020-06-07/20-51-14/"
DIR = "/home/ira/code/projects/rl_trickery/multirun/2020-06-15/16-43-02//"
# DIR = "/home/ira/code/projects/rl_trickery/runs/2020.06.11//"

# DIR = "/home/ira/code/projects/rl_trickery/multirun/0_saved_experiments/low_recurse_depth_maze_13/"

In [5]:
runs = sorted(os.listdir(DIR))
runs

['agent.algo_params.lr=0.0029749749416442157,agent.algo_params.optimizer_type=adam,agent.algo_params.use_timeout=false,agent.network_params.hidden_size=17,agent.num_envs=12,agent.num_steps=8_2',
 'agent.algo_params.lr=0.0029860093695116305,agent.algo_params.optimizer_type=adam,agent.algo_params.use_timeout=true,agent.network_params.hidden_size=40,agent.num_envs=4,agent.num_steps=10_1',
 'agent.algo_params.lr=0.006977898199424571,agent.algo_params.optimizer_type=rmsprop,agent.algo_params.use_timeout=true,agent.network_params.hidden_size=54,agent.num_envs=5,agent.num_steps=9_5',
 'agent.algo_params.lr=0.009368838691960963,agent.algo_params.optimizer_type=adam,agent.algo_params.use_timeout=true,agent.network_params.hidden_size=12,agent.num_envs=15,agent.num_steps=9_3',
 'agent.algo_params.lr=0.01,agent.algo_params.optimizer_type=rmsprop,agent.algo_params.use_timeout=true,agent.network_params.hidden_size=32,agent.num_envs=8,agent.num_steps=10_0',
 'agent.algo_params.lr=0.01107713081412843,

In [6]:
dfs = {}
overrides = {}
for run in runs:
    f_eval = DIR + run + "/eval.csv"
    f_train = DIR + run + "/train.csv"    
    f_override = DIR + run + "/.hydra/overrides.yaml"    
    df_run = pd.read_csv(f_train)
    dfs[run] = df_run
    try:
        with open(f_override) as file:
            overrides[run] = yaml.load(file, Loader=yaml.FullLoader)
    except FileNotFoundError:
            overrides[run] = 0
        

FileNotFoundError: [Errno 2] No such file or directory: '/home/ira/code/projects/rl_trickery/multirun/2020-06-15/16-43-02//agent.algo_params.lr=0.011973827860147845,agent.algo_params.optimizer_type=adam,agent.algo_params.use_timeout=false,agent.network_params.hidden_size=37,agent.num_envs=4,agent.num_steps=8_11/.hydra/overrides.yaml'

In [ ]:
ax = None

target_vars = [
    "episode_reward",
    "loss_entropy",
    "act",
    "fps",
    "episode",
    "value",
]

for i, ovr in enumerate(overrides):
    print(i, ovr)
for target_var in target_vars:
    ax = None
    for i, (key, df) in enumerate(dfs.items()):
        if target_var in df.columns:
            df = df.set_index("timestep")[target_var]
            df = df.rolling(10).mean()
            ax = df.reset_index().plot(x="timestep", y=target_var, label=i, ax=ax, figsize=(10,5))
#             ax = df.reset_index().plot(x="timestep", y=target_var, label=key, ax=ax, figsize=(10,5))
            plt.title(target_var)